In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths #to access dataset

from tensorflow.keras.applications import MobileNetV2 #importing architecture
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input 

from tensorflow.keras.preprocessing.image import ImageDataGenerator #to preprocess image by generating more images
from tensorflow.keras.preprocessing.image import img_to_array #to convert image to array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer #to make model understand the 2 sets of images
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
dataset = '/content/drive/MyDrive/dataset'
imagePaths  = list(paths.list_images(dataset)) #this will make list of the all images

In [4]:
imagePaths

['/content/drive/MyDrive/dataset/with_mask/with_mask477.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask506.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask511.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask586.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask600.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask455.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask454.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask453.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask479.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask512.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask475.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask592.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask001.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask002.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask004.jpg',
 '/content/drive/MyDrive/dataset/with_mask/with_mask003.jpg',
 '/conte

In [5]:
len(imagePaths)

3833

In [ ]:
data = [] #our x
labels = [] #our y

for i in imagePaths:
  label=i.split(os.path.sep)[-2]
  labels.append(label)
  image=load_img(i,target_size=(224,224)) #loaded image with all having size 224
  image=img_to_array(image) #converting images into array
  image=preprocess_input(image) #preprocessing images acc to mobilenet
  data.append(image)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
labels

In [ ]:
data #x

In [ ]:
#converting data and labels into numpy array; they were in list so we converted them into array
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [ ]:
data.shape

(3833, 224, 224, 3)

In [ ]:
data

In [ ]:
labels

array(['with_mask', 'with_mask', 'with_mask', ..., 'without_mask',
       'without_mask', 'without_mask'], dtype='<U12')

In [ ]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [ ]:
labels #now model can understand this

In [ ]:
#split into training and testing
train_X,test_X,train_y,test_y = train_test_split(data,labels,test_size=0.20,random_state=10,stratify=labels)

In [ ]:
train_X.shape

(3066, 224, 224, 3)

In [ ]:
train_y.shape

(3066, 2)

In [ ]:
test_X.shape

(767, 224, 224, 3)

In [ ]:
test_y.shape

(767, 2)

In [ ]:
#generating more number of images;  basically make 6-7 images from 1 image by rotating it and other things

aug=ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [ ]:
#CNN

baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [ ]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dense(32,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

In [ ]:
for layer in baseModel.layers:
  layer.trainable=False

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [ ]:
labels.shape

(3833, 2)

In [ ]:
learning_rate=0.0001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

H=model.fit(
    aug.flow(train_X,train_y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)

model.save('/content/drive/MyDrive/mask1_mobilnet_v2.model')

Epoch 1/20
255/255 [==============================] - 34s 127ms/step - loss: 0.5387 - accuracy: 0.7422 - val_loss: 0.1614 - val_accuracy: 0.9752
Epoch 2/20
255/255 [==============================] - 30s 120ms/step - loss: 0.2638 - accuracy: 0.9170
Epoch 3/20
255/255 [==============================] - 30s 119ms/step - loss: 0.1909 - accuracy: 0.9553
Epoch 4/20
255/255 [==============================] - 31s 120ms/step - loss: 0.1440 - accuracy: 0.9653
Epoch 5/20
255/255 [==============================] - 30s 119ms/step - loss: 0.1212 - accuracy: 0.9691
Epoch 6/20
255/255 [==============================] - 30s 119ms/step - loss: 0.0937 - accuracy: 0.9826
Epoch 7/20
255/255 [==============================] - 31s 121ms/step - loss: 0.1016 - accuracy: 0.9718
Epoch 8/20
255/255 [==============================] - 31s 121ms/step - loss: 0.0940 - accuracy: 0.9774
Epoch 9/20
255/255 [==============================] - 30s 119ms/step - loss: 0.0871 - accuracy: 0.9815
Epoch 10/20
255/255 [==========

In [ ]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_y.argmax(axis=1),predict,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.96      0.99      0.98       383
without_mask       0.99      0.96      0.98       384

    accuracy                           0.98       767
   macro avg       0.98      0.98      0.98       767
weighted avg       0.98      0.98      0.98       767

